In [11]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [18]:
train = pd.read_csv("tcd ml 2019-20 income prediction training (with labels).csv")

test = pd.read_csv("tcd ml 2019-20 income prediction test (without labels).csv")

rename_cols = {"Income in EUR":'Income'}

train = train.rename(columns=rename_cols)

data = pd.concat([train,test],ignore_index=True)

fill_col_dict = {'Year of Record': 1999.0,
 'Gender':'female',
 'Age': 15,
 'Profession': 'principal administrative associate',
 'University Degree': 0,
 'Hair Color': 'Black'}
for col in fill_col_dict.keys():
    data[col] = data[col].fillna(fill_col_dict[col])
    
data.head(20)

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income
0,1,1997.0,0,41.0,Belarus,1239930,steel workers,Bachelor,0,Blond,193,61031.944160
1,2,1996.0,other,41.0,Singapore,1603504,safe event coordinator,Master,0,Black,186,91001.327640
2,3,2018.0,other,28.0,Norway,1298017,receivables/payables analyst,PhD,1,Brown,170,157982.176700
3,4,2006.0,other,33.0,Cuba,751903,fleet assistant,No,1,Black,171,45993.757930
4,5,2010.0,female,46.0,United Arab Emirates,95389,lead trainer,0,0,Blond,188,38022.162170
5,6,1991.0,male,60.0,Liberia,56080,senior security analyst,Bachelor,0,Blond,181,125809.990300
6,7,1982.0,female,71.0,State of Palestine,1205185,program associate,Bachelor,0,Black,174,150319.422600
7,8,2008.0,male,36.0,Israel,31419,retail specialist,0,0,Brown,190,11849.028760
8,9,2015.0,male,43.0,South Sudan,85333,intergovernmental outreach coordinator,No,0,Blond,189,48058.206070
9,10,2019.0,female,16.0,Cuba,1325383,lease analyst,Bachelor,1,Black,157,90940.689120


In [19]:
def create_cat_con(df,cats,cons,normalize=True):
    for i,cat in enumerate(cats):
        vc = df[cat].value_counts(dropna=False, normalize=normalize).to_dict()
        nm = cat + '_FE_FULL'
        df[nm] = df[cat].map(vc)
        df[nm] = df[nm].astype('float32')
        for j,con in enumerate(cons):
#             print("cat %s con %s"%(cat,con))
            new_col = cat +'_'+ con
            print('timeblock frequency encoding:', new_col)
            df[new_col] = df[cat].astype(str)+'_'+df[con].astype(str)
            temp_df = df[new_col]
            fq_encode = temp_df.value_counts(normalize=True).to_dict()
            df[new_col] = df[new_col].map(fq_encode)
            df[new_col] = df[new_col]/df[cat+'_FE_FULL']
    return df

cats = ['Year of Record', 'Gender', 'Country',
        'Profession', 'University Degree','Wears Glasses',
        'Hair Color','Age']
cons = ['Size of City','Body Height [cm]']

data = create_cat_con(data,cats,cons)

for col in train.dtypes[train.dtypes == 'object'].index.tolist():
    feat_le = LabelEncoder()
    feat_le.fit(data[col].unique().astype(str))
    data[col] = feat_le.transform(data[col].astype(str))

del_col = set(['Income','Instance'])
features_col =  list(set(data) - del_col)
features_col

timeblock frequency encoding: Year of Record_Size of City
timeblock frequency encoding: Year of Record_Body Height [cm]
timeblock frequency encoding: Gender_Size of City
timeblock frequency encoding: Gender_Body Height [cm]
timeblock frequency encoding: Country_Size of City
timeblock frequency encoding: Country_Body Height [cm]
timeblock frequency encoding: Profession_Size of City
timeblock frequency encoding: Profession_Body Height [cm]
timeblock frequency encoding: University Degree_Size of City
timeblock frequency encoding: University Degree_Body Height [cm]
timeblock frequency encoding: Wears Glasses_Size of City
timeblock frequency encoding: Wears Glasses_Body Height [cm]
timeblock frequency encoding: Hair Color_Size of City
timeblock frequency encoding: Hair Color_Body Height [cm]
timeblock frequency encoding: Age_Size of City
timeblock frequency encoding: Age_Body Height [cm]


['Country_Size of City',
 'Profession',
 'Hair Color_FE_FULL',
 'Wears Glasses_FE_FULL',
 'University Degree_FE_FULL',
 'Age_FE_FULL',
 'Year of Record',
 'Year of Record_FE_FULL',
 'Gender',
 'Hair Color_Size of City',
 'Hair Color_Body Height [cm]',
 'Country',
 'Wears Glasses_Size of City',
 'Size of City',
 'Wears Glasses_Body Height [cm]',
 'Year of Record_Size of City',
 'University Degree',
 'Age_Size of City',
 'Age',
 'Body Height [cm]',
 'Gender_Body Height [cm]',
 'Year of Record_Body Height [cm]',
 'Country_FE_FULL',
 'Age_Body Height [cm]',
 'Profession_Body Height [cm]',
 'Wears Glasses',
 'Profession_Size of City',
 'Gender_FE_FULL',
 'Hair Color',
 'University Degree_Size of City',
 'Gender_Size of City',
 'University Degree_Body Height [cm]',
 'Country_Body Height [cm]',
 'Profession_FE_FULL']

In [20]:
X_train,X_test = data[features_col].iloc[:111992],data[features_col].iloc[111993:]
Y_train = data['Income'].iloc[:111992]
X_test_id = data['Instance'].iloc[111993:]
x_train,x_val,y_train,y_val = train_test_split(X_train,Y_train,test_size=0.2,random_state=1234)

In [21]:
params = {
          'max_depth': 20,
          'learning_rate': 0.001,
          "boosting": "gbdt",
          "bagging_seed": 11,
          "metric": 'mse',
          "verbosity": -1,
         }
trn_data = lgb.Dataset(x_train, label=y_train)
val_data = lgb.Dataset(x_val, label=y_val)
# test_data = lgb.Dataset(X_test)
clf = lgb.train(params, trn_data, 100000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=500)
pre_test_lgb = clf.predict(X_test)
'done'

Training until validation scores don't improve for 500 rounds
[1000]	training's l2: 6.95933e+09	valid_1's l2: 8.86954e+09
[2000]	training's l2: 3.95107e+09	valid_1's l2: 5.67853e+09
[3000]	training's l2: 3.06048e+09	valid_1's l2: 4.68619e+09
[4000]	training's l2: 2.66399e+09	valid_1's l2: 4.24715e+09
[5000]	training's l2: 2.45656e+09	valid_1's l2: 4.01225e+09
[6000]	training's l2: 2.33637e+09	valid_1's l2: 3.88555e+09
[7000]	training's l2: 2.25498e+09	valid_1's l2: 3.79787e+09
[8000]	training's l2: 2.19426e+09	valid_1's l2: 3.74858e+09
[9000]	training's l2: 2.14765e+09	valid_1's l2: 3.72005e+09
[10000]	training's l2: 2.109e+09	valid_1's l2: 3.70423e+09
[11000]	training's l2: 2.07534e+09	valid_1's l2: 3.68996e+09
[12000]	training's l2: 2.04622e+09	valid_1's l2: 3.68331e+09
[13000]	training's l2: 2.02085e+09	valid_1's l2: 3.67535e+09
[14000]	training's l2: 1.99572e+09	valid_1's l2: 3.66984e+09
[15000]	training's l2: 1.97306e+09	valid_1's l2: 3.66829e+09
[16000]	training's l2: 1.95063e+09

'done'

In [22]:
from sklearn.metrics import mean_squared_error
pre_val_lgb = clf.predict(x_val)
val_mse = mean_squared_error(y_val,pre_val_lgb)
val_rmse = np.sqrt(val_mse)
val_rmse

60518.77307062786

In [9]:
sub_df = pd.DataFrame({'Instance':X_test_id,
                       'Income':pre_test_lgb})
sub_df.head()

,Instance,Income
111993,111994,33669.373574
111994,111995,7200.433756
111995,111996,39625.219363
111996,111997,126474.068067
111997,111998,26196.385173


In [10]:
sub_df.to_csv("sub191015_6.csv",index=False)
'done'

'done'